# Find an Optimized path throguh a Graph using different Algorithms

## Problem solving by Uninformed & Informed Search

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [1]:
#importing libraries
import heapq
import math
import time
import sys
import random
import tracemalloc

In [2]:
#Code Block: Set Initial State (Must handle dynamic inputs)
# Initial Setup
def initialize_state(graph, start, goal):
    return start, goal, set(graph.keys())

In [3]:
#Code Block: Set the matrix for transition & cost (as relevant for the given problem)
campus_map = {
    "admission office": {"hostel office": 2, "library": 4},
    "hostel office": {"admission office": 2, "hostel visit": 2, "canteen": 6, "library": 4},
    "hostel visit": {"hostel office": 2, "canteen": 6, "exit": 4},
    "canteen": {"hostel visit": 6, "hostel office": 6, "library": 7, "dep't visit": 2, "exit": 8},
    "dep't visit": {"canteen": 2, "library": 3, "exit": 5},
    "library": {"admission office": 4, "canteen": 7, "dep't visit": 3, "hostel office": 4},
    "exit": {"dep't visit": 5, "canteen": 8, "hostel visit": 4}
}

# Code Block: Set the matrix for transition & cost
node_coordinates = {
    "admission office": (0, 7),
    "hostel office": (2, 8),
    "hostel visit": (9, 9),
    "canteen": (7, 5),
    "dep't visit": (6, 0),
    "library": (1, 3),
    "exit": (12, 8)
}

In [4]:
#Code Block: Write a function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented
def get_successors(graph, node):
    return graph[node]

In [5]:
#Code block: Write fucntion to handle goal test (Must handle dynamic inputs). Ideally, this would be called while search algorithms are implemented-def hill_climbing_with_restart(graph, start, goal, max_restarts=10):
def is_goal_reached(current_node, goal, visited, all_nodes):
    return visited == all_nodes and current_node == goal

### 2.	Definition of Algorithm 1 (A* algorithm)

In [6]:
#Code Block: Function for algorithm 1 implementation
def euclidean_heuristic(node, target):
    x1, y1 = node_coordinates[node]
    x2, y2 = node_coordinates[target]
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def a_star_visit_all(graph, start, goal):
    tracemalloc.start()
    open_set = []
    all_nodes = set(graph.keys())
    initial_state = (0, start, frozenset([start]), [start], 0)
    heapq.heappush(open_set, initial_state)
    visited_states = {}

    while open_set:
        current_cost, current_node, visited, path, cost_without_heuristic = heapq.heappop(open_set)
        
        if is_goal_reached(current_node, goal, visited, all_nodes):
            memory_used = tracemalloc.get_traced_memory()[1]
            tracemalloc.stop()
            return path, current_cost, cost_without_heuristic, memory_used
        
        state_key = (current_node, visited)
        if state_key in visited_states and visited_states[state_key] <= current_cost:
            continue
        visited_states[state_key] = current_cost

        for neighbor, weight in get_successors(graph, current_node).items():
            new_cost = current_cost + weight
            new_cost_without_heuristic = cost_without_heuristic + weight
            new_visited = visited | frozenset([neighbor])
            heuristic = euclidean_heuristic(neighbor, goal)
            heapq.heappush(open_set, (new_cost + heuristic, neighbor, new_visited, path + [neighbor], new_cost_without_heuristic))
    
    return None, float('inf'), float('inf'), tracemalloc.get_traced_memory()[1]

### 3.	Definition of Algorithm 2 (Random Restart Hill Climbing algorithm)

In [7]:
#Code Block: Function for algorithm 2 implementation
def hill_climbing_with_restart(graph, start, goal, max_restarts=10, max_steps=20):
    tracemalloc.start()
    best_path = None
    best_cost = float('inf')
    random.seed(42)
    nodes_to_visit = list(graph.keys())
    nodes_to_visit.remove(start)
    nodes_to_visit.remove(goal)
    
    for _ in range(max_restarts):
        path = [start] + random.sample(nodes_to_visit, len(nodes_to_visit)) + [goal]
        total_cost = sum(graph[path[i]].get(path[i+1], float('inf')) for i in range(len(path) - 1))
        
        current_path = path
        current_cost = total_cost
        for _ in range(max_steps):
            neighbors = []
            for i in range(1, len(current_path) - 2):
                for j in range(i + 1, len(current_path) - 1):
                    new_path = current_path[:]
                    new_path[i], new_path[j] = new_path[j], new_path[i]
                    new_cost = sum(graph[new_path[k]].get(new_path[k+1], float('inf')) for k in range(len(new_path) - 1))
                    if new_cost < current_cost:
                        neighbors.append((new_path, new_cost))
            
            if not neighbors:
                break

            best_neighbor, best_neighbor_cost = min(neighbors, key=lambda x: x[1])
            current_path = best_neighbor
            current_cost = best_neighbor_cost
        
        if current_cost < best_cost:
            best_cost = current_cost
            best_path = current_path
    
    memory_used = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()
    return best_path, best_cost, memory_used


### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [8]:
#Code Block : Function & call to get inputs (start/end state)
# Code Block: Function & call to get inputs
def get_user_input():
    print("Available locations:")
    for location in campus_map.keys():
        print(f"- {location}")
    
    start_node = input("Enter the starting location: ").strip()
    goal_node = "exit"
    
    if start_node not in campus_map:
        print("Invalid starting location! Please run the program again.")
        sys.exit()
    
    return start_node, goal_node

# Code Block : Function & call to get inputs (start/end state)
start_node, goal_node = get_user_input()


Available locations:
- admission office
- hostel office
- hostel visit
- canteen
- dep't visit
- library
- exit


Enter the starting location:  canteen


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [13]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

# Code Block: Invoke A* Algorithm
# print("Running A*...")
# a_star_path, a_star_cost, a_star_cost_no_heuristic, a_star_memory = a_star_visit_all(campus_map, start_node, goal_node)
# print(f"A* Path: {a_star_path}")
# print(f"A* Cost with Heuristic: {a_star_cost}")
# print(f"A* Cost without Heuristic: {a_star_cost_no_heuristic}")

print("Running A*...")
a_star_start_time = time.time()
a_star_path, a_star_cost, a_star_cost_no_heuristic, a_star_memory = a_star_visit_all(campus_map, start_node, goal_node)
a_star_end_time = time.time()
a_star_runtime = a_star_end_time - a_star_start_time
print(f"A* Path: {a_star_path}")
print(f"A* Cost with Heuristic: {a_star_cost}")
print(f"A* Cost without Heuristic: {a_star_cost_no_heuristic}")



Running A*...
A* Path: ['canteen', "dep't visit", 'library', 'admission office', 'hostel office', 'hostel visit', 'exit']
A* Cost with Heuristic: 64.28691821255524
A* Cost without Heuristic: 17


In [14]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
 # Code Block: Invoke Hill Climbing Algorithm
print("\nRunning Hill Climbing with Restart...")
hill_start_time = time.time()
hill_path, hill_cost, hill_memory = hill_climbing_with_restart(campus_map, start_node, goal_node)
hill_end_time = time.time()
hill_runtime = hill_end_time - hill_start_time
print(f"Hill Climbing Path: {hill_path}")
print(f"Hill Climbing Cost: {hill_cost}")





Running Hill Climbing with Restart...
Hill Climbing Path: ['canteen', "dep't visit", 'library', 'admission office', 'hostel office', 'hostel visit', 'exit']
Hill Climbing Cost: 17


### 5.	Comparitive Analysis (Time and Space Complexity)

In [15]:
#Code Block : Print the Time & Space complexity of algorithm 1
# Code Block : Print the Time & Space complexity of algorithm 1
print("\nA* Algorithm Complexity:")
print("Time Complexity: O(b^d) in worst case \nSpace Complexiity: O(b^d) in worst case")  # A* has exponential worst-case complexity
print(f"Space Occupied: {a_star_memory} bytes")
print(f"Run Time: {a_star_runtime:.6f} seconds")



A* Algorithm Complexity:
Time Complexity: O(b^d) in worst case 
Space Complexiity: O(b^d) in worst case
Space Occupied: 183816 bytes
Run Time: 0.012514 seconds


In [16]:
#Code Block : Print the Time & Space complexity of algorithm 2
# Code Block : Print the Time & Space complexity of algorithm 2
print("\nHill Climbing Algorithm Complexity:")
print("Time Complexity: O(n^2) in worst case with restarts \nSpace Complexiity: O(1)")  # Depends on steps and restarts
print(f"Space Occupied: {hill_memory} bytes")
print(f"Run Time: {hill_runtime:.6f} seconds")


Hill Climbing Algorithm Complexity:
Time Complexity: O(n^2) in worst case with restarts 
Space Complexiity: O(1)
Space Occupied: 1472 bytes
Run Time: 0.005665 seconds


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison: <u>A* guarantees the optimal path but incurs higher space complexity due to its exhaustive search, whereas Hill Climbing with Restart is memory-efficient but may not always find the best solution. Despite A* being more accurate, Hill Climbing executes faster due to its greedy approach. Hence, A* is preferable when memory is not a constraint, while Hill Climbing is suitable for quick approximations in large state spaces.</u>